Based on a code by : https://www.kaggle.com/cstahl12/titanic-with-keras

### Load environment

In [26]:
from __future__ import print_function
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import regularizers
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers import Dense, Activation, Dropout

In [5]:
raw_train = pd.read_csv('./train.csv', index_col = 0)
raw_train['is_test'] = 0
raw_test = pd.read_csv('./test.csv', index_col = 0)
raw_test['is_test'] = 1

In [6]:
all_data = pd.concat((raw_train,raw_test),axis=0)

### Functions to preprocess the data

In [9]:
def get_title_last_name(name):
    full_name = name.str.split(',', n=0, expand=True)
    last_name = full_name[0]
    titles = full_name[1].str.split('.', n=0, expand=True)
    titles = titles[0]
    return(titles)

def get_titles_from_names(df):
    df['Title'] = get_title_last_name(df['Name'])
    df = df.drop(['Name'], axis=1)
    return(df)

def get_dummy_cats(df):
    return(pd.get_dummies(df,columns=['Title','Pclass','Sex','Embarked','Cabin','Cabin_letter']))

def get_cabin_letter(df):
    df['Cabin'].fillna('Z', inplace=True)
    df['Cabin_letter'] = df['Cabin'].str[0]
    return(df)

def further_process(df):
    #family size
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    #has cabin
    #df['Has_Cabin'] = df["Cabin"].apply(lambda x: 0 if type(x) == float else 1)
    
    #is alone
    df['IsAlone'] = 0
    df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
    
    return(df)

def process_data(df):
    #preprocess titles, cabin, embarked
    df = get_titles_from_names(df)
    df['Embarked'].fillna('S', inplace=True)
    df=get_cabin_letter(df)
    
    #drop remaining features
    df = df.drop(['Ticket','Fare'], axis=1)
    
    #create dummy values for categorical features
    df = get_dummy_cats(df)
    
    #further process based on https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python
    df = further_process(df)

    
    return(df)

proc_data = process_data(all_data)
proc_train = proc_data[proc_data['is_test'] == 0]
proc_test = proc_data[proc_data['is_test'] == 1] 

In [10]:
proc_data.head()

,Age,Parch,SibSp,Survived,is_test,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_Z,FamilySize,IsAlone
PassengerId,,,,,,,,,,,,,,,,,,,,,
1,22.0,0,1,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,0
2,38.0,0,1,1.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,0
3,26.0,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,35.0,0,1,1.0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,2,0
5,35.0,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


### Build Network to predict missing ages

In [11]:
for_age_train = proc_data.drop(['Survived', 'is_test'], axis = 1).dropna(axis=0)
X_train_age=for_age_train.drop('Age', axis=1)
y_train_age=for_age_train['Age']



In [17]:
#create model
tmodel = Sequential()
tmodel.add(Dense(input_dim=X_train_age.shape[1], units=128,
                kernel_initializer='normal',
                bias_initializer='zeros'))
tmodel.add(Activation('relu'))

for i in range(0,8):
    tmodel.add(Dense(units=64,kernel_initializer='he_normal',
                   bias_initializer='zeros'))
    tmodel.add(Activation('relu'))
    tmodel.add(Dropout(.25))
    
tmodel.add(Dense(units=1))
tmodel.add(Activation('linear'))

tmodel.compile(loss='mean_squared_error', optimizer='adam')


In [18]:
tmodel.fit(X_train_age.values, y_train_age.values, epochs=1000, verbose=2,validation_split=0.33)

Train on 700 samples, validate on 346 samples
Epoch 1/1000
0s - loss: 843.8806 - val_loss: 297.1605
Epoch 2/1000
0s - loss: 370.1098 - val_loss: 271.6745
Epoch 3/1000
0s - loss: 251.4862 - val_loss: 255.6224
Epoch 4/1000
0s - loss: 256.5717 - val_loss: 397.1451
Epoch 5/1000
0s - loss: 216.4065 - val_loss: 361.7270
Epoch 6/1000
0s - loss: 195.4814 - val_loss: 280.8828
Epoch 7/1000
0s - loss: 190.0613 - val_loss: 343.0722
Epoch 8/1000
0s - loss: 190.7573 - val_loss: 309.1142
Epoch 9/1000
0s - loss: 173.8453 - val_loss: 369.4911
Epoch 10/1000
0s - loss: 171.2814 - val_loss: 330.6114
Epoch 11/1000
0s - loss: 167.1759 - val_loss: 256.4775
Epoch 12/1000
0s - loss: 167.7201 - val_loss: 359.1294
Epoch 13/1000
0s - loss: 163.3484 - val_loss: 289.9714
Epoch 14/1000
0s - loss: 140.5451 - val_loss: 258.1243
Epoch 15/1000
0s - loss: 149.7967 - val_loss: 278.7168
Epoch 16/1000
0s - loss: 153.5194 - val_loss: 308.0934
Epoch 17/1000
0s - loss: 144.5584 - val_loss: 324.5223
Epoch 18/1000
0s - loss: 143

0s - loss: 99.3235 - val_loss: 142.5512
Epoch 151/1000
0s - loss: 105.2437 - val_loss: 156.1939
Epoch 152/1000
0s - loss: 92.9550 - val_loss: 165.5416
Epoch 153/1000
0s - loss: 98.0542 - val_loss: 156.2250
Epoch 154/1000
0s - loss: 94.8108 - val_loss: 153.7856
Epoch 155/1000
0s - loss: 97.1953 - val_loss: 155.7253
Epoch 156/1000
0s - loss: 99.7413 - val_loss: 163.5480
Epoch 157/1000
0s - loss: 100.2154 - val_loss: 173.0246
Epoch 158/1000
0s - loss: 100.0800 - val_loss: 133.4349
Epoch 159/1000
0s - loss: 101.6262 - val_loss: 161.7715
Epoch 160/1000
0s - loss: 100.4674 - val_loss: 167.7624
Epoch 161/1000
0s - loss: 100.4890 - val_loss: 173.7594
Epoch 162/1000
0s - loss: 104.2256 - val_loss: 171.1466
Epoch 163/1000
0s - loss: 95.2164 - val_loss: 153.0684
Epoch 164/1000
0s - loss: 96.6282 - val_loss: 152.3881
Epoch 165/1000
0s - loss: 96.5470 - val_loss: 156.3694
Epoch 166/1000
0s - loss: 92.3247 - val_loss: 165.8516
Epoch 167/1000
0s - loss: 100.1984 - val_loss: 156.4729
Epoch 168/1000
0s

0s - loss: 96.2242 - val_loss: 151.1502
Epoch 300/1000
0s - loss: 96.6686 - val_loss: 145.1844
Epoch 301/1000
0s - loss: 92.4373 - val_loss: 160.7929
Epoch 302/1000
0s - loss: 89.1551 - val_loss: 172.6859
Epoch 303/1000
0s - loss: 89.7886 - val_loss: 155.8354
Epoch 304/1000
0s - loss: 97.0922 - val_loss: 140.8091
Epoch 305/1000
0s - loss: 94.3857 - val_loss: 152.2636
Epoch 306/1000
0s - loss: 94.6704 - val_loss: 143.2134
Epoch 307/1000
0s - loss: 88.7171 - val_loss: 157.3972
Epoch 308/1000
0s - loss: 86.5666 - val_loss: 139.1212
Epoch 309/1000
0s - loss: 97.1483 - val_loss: 181.9605
Epoch 310/1000
0s - loss: 94.8078 - val_loss: 178.4574
Epoch 311/1000
0s - loss: 92.7944 - val_loss: 171.6165
Epoch 312/1000
0s - loss: 97.2660 - val_loss: 177.3433
Epoch 313/1000
0s - loss: 96.1417 - val_loss: 167.6310
Epoch 314/1000
0s - loss: 91.6356 - val_loss: 165.1436
Epoch 315/1000
0s - loss: 93.9818 - val_loss: 160.8283
Epoch 316/1000
0s - loss: 90.6508 - val_loss: 163.2713
Epoch 317/1000
0s - loss:

0s - loss: 89.7265 - val_loss: 176.9558
Epoch 449/1000
0s - loss: 90.6931 - val_loss: 177.9286
Epoch 450/1000
0s - loss: 94.3941 - val_loss: 176.2285
Epoch 451/1000
0s - loss: 89.5880 - val_loss: 180.9026
Epoch 452/1000
0s - loss: 90.2161 - val_loss: 168.5601
Epoch 453/1000
0s - loss: 88.5207 - val_loss: 153.5906
Epoch 454/1000
0s - loss: 88.6866 - val_loss: 176.9349
Epoch 455/1000
0s - loss: 99.5330 - val_loss: 180.4015
Epoch 456/1000
0s - loss: 92.5026 - val_loss: 177.3023
Epoch 457/1000
0s - loss: 89.8592 - val_loss: 173.5947
Epoch 458/1000
0s - loss: 90.6177 - val_loss: 167.4885
Epoch 459/1000
0s - loss: 97.3233 - val_loss: 166.0848
Epoch 460/1000
0s - loss: 84.5751 - val_loss: 175.4443
Epoch 461/1000
0s - loss: 98.2644 - val_loss: 144.5466
Epoch 462/1000
0s - loss: 90.7405 - val_loss: 161.4531
Epoch 463/1000
0s - loss: 90.7428 - val_loss: 174.6793
Epoch 464/1000
0s - loss: 86.8776 - val_loss: 161.6857
Epoch 465/1000
0s - loss: 91.3039 - val_loss: 137.9230
Epoch 466/1000
0s - loss:

0s - loss: 87.9691 - val_loss: 169.2758
Epoch 599/1000
0s - loss: 87.7392 - val_loss: 166.1404
Epoch 600/1000
0s - loss: 82.8957 - val_loss: 162.3718
Epoch 601/1000
0s - loss: 84.7130 - val_loss: 165.8592
Epoch 602/1000
0s - loss: 87.8568 - val_loss: 179.9272
Epoch 603/1000
0s - loss: 83.9783 - val_loss: 180.6091
Epoch 604/1000
0s - loss: 84.0949 - val_loss: 178.3697
Epoch 605/1000
0s - loss: 90.3967 - val_loss: 159.8755
Epoch 606/1000
0s - loss: 88.0511 - val_loss: 160.4514
Epoch 607/1000
0s - loss: 86.6151 - val_loss: 173.0536
Epoch 608/1000
0s - loss: 92.5344 - val_loss: 163.5174
Epoch 609/1000
0s - loss: 91.6865 - val_loss: 167.0298
Epoch 610/1000
0s - loss: 85.1754 - val_loss: 159.1269
Epoch 611/1000
0s - loss: 85.4599 - val_loss: 163.2847
Epoch 612/1000
0s - loss: 87.1403 - val_loss: 184.3353
Epoch 613/1000
0s - loss: 92.4687 - val_loss: 183.0382
Epoch 614/1000
0s - loss: 85.4057 - val_loss: 170.5077
Epoch 615/1000
0s - loss: 89.1617 - val_loss: 177.8045
Epoch 616/1000
0s - loss:

0s - loss: 90.2543 - val_loss: 151.0098
Epoch 748/1000
0s - loss: 99.1583 - val_loss: 173.1825
Epoch 749/1000
0s - loss: 87.2151 - val_loss: 179.6101
Epoch 750/1000
0s - loss: 85.7598 - val_loss: 179.5110
Epoch 751/1000
0s - loss: 85.0654 - val_loss: 177.2005
Epoch 752/1000
0s - loss: 80.3073 - val_loss: 176.0222
Epoch 753/1000
0s - loss: 81.4844 - val_loss: 167.0391
Epoch 754/1000
0s - loss: 89.3227 - val_loss: 170.0167
Epoch 755/1000
0s - loss: 82.6746 - val_loss: 155.3925
Epoch 756/1000
0s - loss: 88.3078 - val_loss: 164.0591
Epoch 757/1000
0s - loss: 87.5286 - val_loss: 186.6767
Epoch 758/1000
0s - loss: 82.2064 - val_loss: 188.3606
Epoch 759/1000
0s - loss: 88.2776 - val_loss: 187.0512
Epoch 760/1000
0s - loss: 80.4975 - val_loss: 192.9458
Epoch 761/1000
0s - loss: 87.0113 - val_loss: 197.9989
Epoch 762/1000
0s - loss: 87.7104 - val_loss: 186.1232
Epoch 763/1000
0s - loss: 83.8097 - val_loss: 179.1162
Epoch 764/1000
0s - loss: 83.8674 - val_loss: 169.0847
Epoch 765/1000
0s - loss:

0s - loss: 85.7216 - val_loss: 177.6144
Epoch 899/1000
0s - loss: 86.9341 - val_loss: 171.5064
Epoch 900/1000
0s - loss: 80.2325 - val_loss: 181.2017
Epoch 901/1000
0s - loss: 83.1509 - val_loss: 178.6862
Epoch 902/1000
0s - loss: 86.6915 - val_loss: 174.7728
Epoch 903/1000
0s - loss: 83.4886 - val_loss: 166.5996
Epoch 904/1000
0s - loss: 82.3810 - val_loss: 171.8698
Epoch 905/1000
0s - loss: 85.8782 - val_loss: 177.4955
Epoch 906/1000
0s - loss: 81.6070 - val_loss: 188.2141
Epoch 907/1000
0s - loss: 85.1791 - val_loss: 184.7789
Epoch 908/1000
0s - loss: 85.7839 - val_loss: 178.5000
Epoch 909/1000
0s - loss: 88.7243 - val_loss: 176.8319
Epoch 910/1000
0s - loss: 83.9818 - val_loss: 172.1096
Epoch 911/1000
0s - loss: 80.1728 - val_loss: 178.9928
Epoch 912/1000
0s - loss: 85.2352 - val_loss: 179.0793
Epoch 913/1000
0s - loss: 86.4264 - val_loss: 176.9532
Epoch 914/1000
0s - loss: 86.6907 - val_loss: 191.9460
Epoch 915/1000
0s - loss: 81.5801 - val_loss: 161.9161
Epoch 916/1000
0s - loss:

In [19]:
train_data = proc_train
train_data.loc[train_data['Age'].isnull()]

,Age,Parch,SibSp,Survived,is_test,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_Z,FamilySize,IsAlone
PassengerId,,,,,,,,,,,,,,,,,,,,,
6,NaN,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
18,NaN,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
20,NaN,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
27,NaN,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
29,NaN,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
30,NaN,0,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
32,NaN,0,1,1.0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,2,0
33,NaN,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
37,NaN,0,0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


In [20]:
to_pred = train_data.loc[train_data['Age'].isnull()].drop(['Age','Survived','is_test'],axis=1)
p = tmodel.predict(to_pred.values)
train_data['Age'].loc[train_data['Age'].isnull()] = p

/home/ikanez/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
test_data = proc_test
to_pred = test_data.loc[test_data['Age'].isnull()].drop(['Age','Survived','is_test'],axis=1)
p = tmodel.predict(to_pred.values)
test_data['Age'].loc[test_data['Age'].isnull()] = p

/home/ikanez/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/ikanez/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [22]:
train_data.loc[train_data['Age'].isnull()]

,Age,Parch,SibSp,Survived,is_test,Title_ Capt,Title_ Col,Title_ Don,Title_ Dona,Title_ Dr,...,Cabin_letter_B,Cabin_letter_C,Cabin_letter_D,Cabin_letter_E,Cabin_letter_F,Cabin_letter_G,Cabin_letter_T,Cabin_letter_Z,FamilySize,IsAlone
PassengerId,,,,,,,,,,,,,,,,,,,,,


In [23]:
y = pd.get_dummies(train_data['Survived'])
y.head()

,0.0,1.0
PassengerId,,
1,1,0
2,0,1
3,0,1
4,0,1
5,1,0


In [24]:
X = train_data.drop(['Survived','is_test'], axis=1)

In [29]:
#create model

#titanic4 (0.7799)
model = Sequential()
model.add(Dense(input_dim=X.shape[1],units=128, kernel_initializer='he_normal', bias_initializer='zeros'))
model.add(Activation('relu'))

for i in range(0,8):
    model.add(Dense(units=384, kernel_initializer='he_normal', bias_initializer='zeros'))
    model.add(Activation('relu'))
    model.add(Dropout(.40))

model.add(Dense(units=2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
model.fit(X.values, y.values, epochs=300,validation_split=0.4, batch_size=30, verbose=1, shuffle=True)

Train on 534 samples, validate on 357 samples
Epoch 1/300
534/534 [==============================] - 0s - loss: 0.2958 - acc: 0.8839 - val_loss: 0.6514 - val_acc: 0.8067
Epoch 2/300
534/534 [==============================] - 0s - loss: 0.2880 - acc: 0.8876 - val_loss: 0.5940 - val_acc: 0.8207
Epoch 3/300
534/534 [==============================] - 0s - loss: 0.2604 - acc: 0.9045 - val_loss: 0.7023 - val_acc: 0.8179
Epoch 4/300
534/534 [==============================] - 0s - loss: 0.2645 - acc: 0.9026 - val_loss: 0.7044 - val_acc: 0.8179
Epoch 5/300
534/534 [==============================] - 0s - loss: 0.2684 - acc: 0.8989 - val_loss: 0.6718 - val_acc: 0.8179
Epoch 6/300
534/534 [==============================] - 0s - loss: 0.2595 - acc: 0.9026 - val_loss: 0.7237 - val_acc: 0.8207
Epoch 7/300
534/534 [==============================] - 0s - loss: 0.2631 - acc: 0.9007 - val_loss: 0.7310 - val_acc: 0.8235
Epoch 8/300
534/534 [==============================] - 0s - loss: 0.2703 - acc: 0.8970

534/534 [==============================] - 0s - loss: 0.2401 - acc: 0.9007 - val_loss: 1.2734 - val_acc: 0.8235
Epoch 67/300
534/534 [==============================] - 0s - loss: 0.2327 - acc: 0.9045 - val_loss: 1.3824 - val_acc: 0.8179
Epoch 68/300
534/534 [==============================] - 0s - loss: 0.2647 - acc: 0.8989 - val_loss: 1.1664 - val_acc: 0.8179
Epoch 69/300
534/534 [==============================] - 0s - loss: 0.2832 - acc: 0.9045 - val_loss: 1.2383 - val_acc: 0.8123
Epoch 70/300
534/534 [==============================] - 0s - loss: 0.2424 - acc: 0.9064 - val_loss: 1.3342 - val_acc: 0.8179
Epoch 71/300
534/534 [==============================] - 0s - loss: 0.2350 - acc: 0.9139 - val_loss: 1.4122 - val_acc: 0.8207
Epoch 72/300
534/534 [==============================] - 0s - loss: 0.2523 - acc: 0.9082 - val_loss: 1.2898 - val_acc: 0.8151
Epoch 73/300
534/534 [==============================] - 0s - loss: 0.2495 - acc: 0.9026 - val_loss: 1.4413 - val_acc: 0.8179
Epoch 74/300


534/534 [==============================] - 0s - loss: 0.3306 - acc: 0.8895 - val_loss: 1.7316 - val_acc: 0.8011
Epoch 132/300
534/534 [==============================] - 0s - loss: 0.3371 - acc: 0.8783 - val_loss: 0.9164 - val_acc: 0.8207
Epoch 133/300
534/534 [==============================] - 0s - loss: 0.2695 - acc: 0.8933 - val_loss: 1.1870 - val_acc: 0.8151
Epoch 134/300
534/534 [==============================] - 0s - loss: 0.2641 - acc: 0.9007 - val_loss: 0.9445 - val_acc: 0.8291
Epoch 135/300
534/534 [==============================] - 0s - loss: 0.2812 - acc: 0.9007 - val_loss: 0.9143 - val_acc: 0.8095
Epoch 136/300
534/534 [==============================] - 0s - loss: 0.2679 - acc: 0.8933 - val_loss: 0.9553 - val_acc: 0.8151
Epoch 137/300
534/534 [==============================] - 0s - loss: 0.2535 - acc: 0.9007 - val_loss: 1.2138 - val_acc: 0.8095
Epoch 138/300
534/534 [==============================] - 0s - loss: 0.2393 - acc: 0.9007 - val_loss: 1.2669 - val_acc: 0.8095
Epoch 

534/534 [==============================] - 0s - loss: 0.2284 - acc: 0.9101 - val_loss: 1.6774 - val_acc: 0.8151
Epoch 197/300
534/534 [==============================] - 0s - loss: 0.2438 - acc: 0.9007 - val_loss: 1.3568 - val_acc: 0.8235
Epoch 198/300
534/534 [==============================] - 0s - loss: 0.2288 - acc: 0.9082 - val_loss: 1.5039 - val_acc: 0.8123
Epoch 199/300
534/534 [==============================] - 0s - loss: 0.2249 - acc: 0.9082 - val_loss: 1.8509 - val_acc: 0.8095
Epoch 200/300
534/534 [==============================] - 0s - loss: 0.2210 - acc: 0.9120 - val_loss: 1.8904 - val_acc: 0.8179
Epoch 201/300
534/534 [==============================] - 0s - loss: 0.2219 - acc: 0.9120 - val_loss: 1.9484 - val_acc: 0.8179
Epoch 202/300
534/534 [==============================] - 0s - loss: 0.2498 - acc: 0.9101 - val_loss: 1.3800 - val_acc: 0.8123
Epoch 203/300
534/534 [==============================] - 0s - loss: 0.2369 - acc: 0.9139 - val_loss: 1.4120 - val_acc: 0.8179
Epoch 

534/534 [==============================] - 0s - loss: 0.2443 - acc: 0.9120 - val_loss: 1.2300 - val_acc: 0.8179
Epoch 262/300
534/534 [==============================] - 0s - loss: 0.2488 - acc: 0.9045 - val_loss: 1.3799 - val_acc: 0.8235
Epoch 263/300
534/534 [==============================] - 0s - loss: 0.2454 - acc: 0.9101 - val_loss: 1.3496 - val_acc: 0.8179
Epoch 264/300
534/534 [==============================] - 0s - loss: 0.2449 - acc: 0.9064 - val_loss: 1.3587 - val_acc: 0.8207
Epoch 265/300
534/534 [==============================] - 0s - loss: 0.2726 - acc: 0.9026 - val_loss: 1.3834 - val_acc: 0.8067
Epoch 266/300
534/534 [==============================] - 0s - loss: 0.2593 - acc: 0.8858 - val_loss: 1.1528 - val_acc: 0.8067
Epoch 267/300
534/534 [==============================] - 0s - loss: 0.2554 - acc: 0.8858 - val_loss: 0.9997 - val_acc: 0.8123
Epoch 268/300
534/534 [==============================] - 0s - loss: 0.2541 - acc: 0.8989 - val_loss: 1.0648 - val_acc: 0.8179
Epoch 

In [24]:
model

In [25]:
test_data.columns

Index(['Age', 'Parch', 'SibSp', 'Survived', 'is_test', 'Title_ Capt',
       'Title_ Col', 'Title_ Don', 'Title_ Dona', 'Title_ Dr',
       ...
       'Cabin_Z', 'Cabin_letter_A', 'Cabin_letter_B', 'Cabin_letter_C',
       'Cabin_letter_D', 'Cabin_letter_E', 'Cabin_letter_F', 'Cabin_letter_G',
       'Cabin_letter_T', 'Cabin_letter_Z'],
      dtype='object', length=227)

In [32]:
p_survived = model.predict_classes(test_data.drop(['Survived','is_test'],axis = 1).values)




 32/418 [=>............................] - ETA: 1s

In [33]:
submission = pd.DataFrame()
submission['PassengerId'] = test_data.index
submission['Survived'] = p_survived

In [34]:
submission.shape

(418, 2)

In [35]:
submission.to_csv('titanic_keras6.csv', index=False)